# *btest* initial steps

step-by-step

In [1]:
import os
import time
import numpy as np
import pandas as pd
from btest import utils
from btest import blockplot
from scipy.stats import t

In [2]:
method = 'spearman'
min_var = 0.0
fdr =0.1

In [3]:
dataX , dataY, featuresX, featuresY  = utils.readData('../demo/X_dataset.txt', '../demo/Y_dataset.txt', min_var=min_var)

Dataset X dimension  after cleaning:  (100, 50)
Dataset Y dimension  after cleaning:  (100, 50)


In [4]:
%%time
within_X = utils.btest_corr(dataX, featuresX, method=method, fdr=fdr, Type='withinX')
within_Y  = utils.btest_corr(dataY, featuresY, method=method, fdr=fdr, Type='withinY')
dataAll = np.concatenate((dataX, dataY), axis=0)
X_Y = utils.btest_corr(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

CPU times: user 4.65 s, sys: 11.3 ms, total: 4.66 s
Wall time: 4.66 s


In [5]:
X_Y.head()

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
165,X1,Y66,0.021416,0.327972,49,212.013466,0.65493,X_Y
174,X1,Y75,0.033062,0.305052,49,163.658486,0.65493,X_Y
2747,X27,Y74,0.036443,0.296654,50,120.262541,0.65493,X_Y
173,X1,Y74,0.040235,0.294122,49,99.582635,0.65493,X_Y
1526,X15,Y41,0.042550,0.287992,50,84.249800,0.65493,X_Y


In [15]:
nas = np.isnan(dataAll[0]+dataAll[1])
not_na = sum(~nas)
not_na

49

In [6]:
X_Y[(X_Y['Feature_1']=='X0')&(X_Y['Feature_2']=='Y66')]

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
66,X0,Y66,0.119186,0.223218,50,5.021027,0.65493,X_Y


In [34]:
%%time
X_Y_3 = utils.btest_corr_3(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

CPU times: user 143 ms, sys: 6.87 ms, total: 150 ms
Wall time: 153 ms


/Users/mbagh/opt/anaconda3/envs/btest_env/lib/python3.8/site-packages/btest/utils.py:299: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_f.loc[:, 't_statistic'] = (df_f.loc[:, 'Correlation']*
/Users/mbagh/opt/anaconda3/envs/btest_env/lib/python3.8/site-packages/btest/utils.py:303: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_f.loc[:, 'pval'] = 2 * (1 - t.cdf(abs(df_f.loc[:, 't_statistic']),
/Users/mbagh/opt/anaconda3/envs/btest_env/lib/python3.8/site-packages/btest/utils.py:310: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will

In [35]:
X_Y_3.head()

,Feature_1,Feature_2,Correlation,complete_obs,t_statistic,pval,P_adjusted,bh_fdr_threshold,Type
19872,Y92,Y93,0.978539,50.0,32.900503,0.0,0.0,0.016883,Y_Y
4500,X24,X25,0.971709,49.0,28.206115,0.0,0.0,0.016883,X_X
15170,Y2,Y26,0.969082,50.0,27.211086,0.0,0.0,0.016883,Y_Y
7216,X40,X77,0.968504,50.0,26.947950,0.0,0.0,0.016883,X_X
420,X2,X26,0.966043,50.0,25.903472,0.0,0.0,0.016883,X_X


In [43]:
err = []
for i in range(X_Y.shape[0]):
    var1 , var2 = X_Y.iloc[i, [0,1]]
    a = X_Y.loc[(X_Y.loc[:,'Feature_1']==var1) & (X_Y.loc[:,'Feature_2']==var2), 'Not_NAs'].values[0]
    b = X_Y_3.loc[(X_Y_3.loc[:,'Feature_1']==var1) & (X_Y_3.loc[:,'Feature_2']==var2), 'complete_obs'].values[0]
    if a != b:
        err.append([var1, var2])

In [44]:
err

[]

In [9]:
X_Y_3[(X_Y_3['Feature_1']=='X0')&(X_Y_3['Feature_2']=='Y66')]

,Feature_1,Feature_2,Correlation,complete_obs,t_statistic,pval,P_adjusted,bh_fdr_threshold,Type
165,X0,Y66,0.223218,50.0,1.586529,0.119186,0.588975,0.016883,X_Y


In [10]:
X_Y_3.loc[:,'Type'].value_counts()

X_Y    10000
Y_Y     4950
X_X     4950
Name: Type, dtype: int64

In [11]:
X_Y[(X_Y['Feature_1']=='X0')&(X_Y['Feature_2']=='Y66')]

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
66,X0,Y66,0.119186,0.223218,50,5.021027,0.65493,X_Y


In [13]:
X_Y_32 = X_Y_3[X_Y_3["Type"] == "X_Y"]

In [14]:
X_Y_32

,Feature_1,Feature_2,Correlation,complete_obs,t_statistic,pval,P_adjusted,bh_fdr_threshold,Type
363,X1,Y66,0.327972,49.0,2.380114,0.021416,0.124030,0.016883,X_Y
372,X1,Y75,0.305052,49.0,2.195998,0.033062,0.190266,0.016883,X_Y
5168,X27,Y74,0.296654,50.0,2.152155,0.036443,0.209480,0.016883,X_Y
371,X1,Y74,0.294122,49.0,2.109718,0.040235,0.230745,0.016883,X_Y
3005,X15,Y41,0.287992,50.0,2.083542,0.042550,0.243530,0.016883,X_Y
...,...,...,...,...,...,...,...,...,...
12466,X77,Y47,-0.000098,50.0,-0.000682,0.999459,0.999710,0.016883,X_Y
4317,X22,Y93,0.000049,50.0,0.000341,0.999729,0.999930,0.016883,X_Y
9223,X53,Y8,0.000049,50.0,0.000341,0.999729,0.999880,0.016883,X_Y
13618,X87,Y34,0.000049,50.0,0.000341,0.999729,0.999830,0.016883,X_Y


In [15]:
simtable = pd.pivot(X_Y_32, index="Feature_1", columns="Feature_2", values='Correlation') #Reshape from long to wide

In [16]:
simtable

Feature_2,Y0,Y1,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,...,Y90,Y91,Y92,Y93,Y94,Y95,Y96,Y97,Y98,Y99
Feature_1,,,,,,,,,,,,,,,,,,,,,
X0,-0.064777,-0.028155,-0.032880,0.028549,-0.008071,-0.045866,-0.010041,0.013192,0.002166,0.034849,...,0.005956,-0.081299,-0.072638,-0.075015,-0.124385,-0.060199,-0.068701,-0.081299,-0.072638,-0.101004
X1,-0.166641,-0.132381,-0.074402,0.001543,-0.091251,-0.115725,-0.077985,-0.059914,-0.071081,-0.019379,...,0.127439,0.014537,0.040030,0.042889,-0.020316,0.069628,-0.008864,0.035455,0.033337,0.002354
X10,-0.093670,-0.071618,-0.065859,-0.001920,-0.071850,-0.086811,-0.020427,0.004922,-0.028303,-0.002313,...,0.104007,0.000591,0.004528,0.008712,-0.042331,0.028155,-0.024606,-0.009646,0.010827,-0.020033
X11,-0.140136,-0.079888,-0.078559,-0.010189,-0.053740,-0.104134,-0.060986,-0.038000,-0.068862,0.015407,...,0.022642,-0.068701,-0.056102,-0.057836,-0.101792,-0.042479,-0.068701,-0.077362,-0.062402,-0.102185
X12,-0.089048,-0.067877,-0.021654,0.049724,-0.054123,-0.058359,-0.035259,0.002249,-0.028186,0.035913,...,0.007453,-0.107201,-0.089578,-0.096815,-0.148073,-0.073749,-0.113476,-0.104534,-0.108037,-0.137505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X95,-0.082004,-0.055867,-0.035145,-0.088551,-0.134843,0.049409,-0.119659,-0.095245,-0.055621,-0.171097,...,0.240992,0.174803,0.262992,0.215397,0.207915,0.164206,0.237795,0.181102,0.225197,0.154361
X96,0.004627,0.018557,0.032979,-0.033816,-0.034055,0.112402,-0.051536,0.010681,0.031404,-0.096673,...,0.208703,0.162205,0.237795,0.185470,0.201614,0.109077,0.218898,0.155906,0.231496,0.140973
X97,-0.010730,-0.033422,0.025103,-0.060199,-0.080512,0.097441,-0.084613,-0.035785,-0.007580,-0.142351,...,0.214609,0.118110,0.218898,0.183894,0.189407,0.151998,0.187402,0.137008,0.193701,0.109470


In [12]:
t1 = time.time()
for i in range(1000):
    i
t2 = time.time()
print(t2-t1)

8.130073547363281e-05


In [13]:
nrow = 5000
ncol = 100
aa = np.array(np.random.random(size=nrow*ncol), dtype=np.float32)
aa[np.random.randint(0, nrow*ncol, size = int(nrow*ncol/20))]=np.nan
aa = aa.reshape((nrow,ncol))
feature_names = ['P'+str(i) for i in range(nrow)]

In [14]:
%%time
#test
test_withinX = utils.btest_corr_3(aa, features=feature_names, method=method, fdr=fdr, Type='withinX')

KeyboardInterrupt: 

In [15]:
%%time
test_withinX = utils.btest_corr_3(aa, features=feature_names, method='pearson', fdr=fdr, Type='withinX')

correlation time:  3.98840594291687
obs count time:  1.3174548149108887
long transform time:  4.27480411529541
p-value time:  5.411080837249756
bh time:  4.489036798477173
names time:  0.045684814453125


KeyboardInterrupt: 

In [16]:
test_withinX.head()

,Feature_1,Feature_2,Correlation,complete_obs,t_statistic,pval,P_adjusted,bh_fdr_threshold,Type
4305936,P951,P4564,0.539350,88.0,5.939717,5.898084e-08,0.737113,0.994688,withinX
9689339,P2629,P4104,-0.528004,87.0,-5.732120,1.475354e-07,0.921912,0.994688,withinX
8107581,P2036,P3284,-0.515333,91.0,-5.672923,1.723654e-07,0.718045,0.994688,withinX
2579194,P545,P3525,0.521917,86.0,5.607817,2.557842e-07,0.799166,0.994688,withinX
8059819,P2020,P3050,0.507380,91.0,5.554705,2.854661e-07,0.713523,0.994688,withinX


In [17]:
df_test = pd.DataFrame(aa.T)

In [18]:
%%time
cr_test = df_test.corr(method = 'spearman')

CPU times: user 15min 18s, sys: 8.09 s, total: 15min 26s
Wall time: 15min 46s


In [ ]:
X_Y[(X_Y['Feature_1']=='X0')&(X_Y['Feature_2']=='Y66')]

In [ ]:
%%time
X_Y_3 = utils.btest_corr_3(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')

In [ ]:
for i in iRow:
    for j in iCol:
        if i<=j:
            #print(i, j)
            #print(dataAll[i])
            X = dataAll[i]
            Y = dataAll[j]
            nas = np.logical_or(X != X, Y != Y)
            not_na = sum(~nas)
            #X = Y[~nas]
            #Y = Y[~nas]
            #new_X, new_Y = remove_pairs_with_a_missing(X, Y)
            correlation = cr[i, j]
            pval =  pv[i, j]
            tests.append([features[i], features_y[j], pval, correlation, not_na])
results = pd.DataFrame(tests, columns=['Feature_1','Feature_2','pval', 'Correlation', 'Not_NAs'])

p_adust, p_threshold = utils.bh(results["pval"].values, fdr)

results["P_adusted"] = p_adust
results["bh_fdr_threshold"] = p_threshold
results['Type'] = Type
results = results.sort_values(['pval', 'Correlation'],
                              ascending=[True, False])

In [ ]:
X_Y_3[(X_Y_3['Feature_1']=='X0')&(X_Y_3['Feature_2']=='Y66')]

In [ ]:
X_Y_3.loc[:,'Type'].value_counts()

In [ ]:
# creating the complete dataset
#dataAll_df = pd.DataFrame(dataAll.T, columns=featuresAll)

In [ ]:
X_Y_3.loc[:,'Type'] = X_Y_3.Feature_1.str[-1]+X_Y_3.Feature_2.str[-2:]

In [ ]:
X_Y_3.head()

In [ ]:
dataAll = dataX
features = featuresX
features_y = None
Type = 'withinX'
iRow = list(range(0, len(features)))
if Type == 'X_Y':
    iCol = list(range(len(features), len(features)+len(features_y)-1))
    features_y = features + features_y
else:
    features_y = features
    iCol = list(range(len(features)))
tests = []

# creating the complete dataset
dataAll2 = pd.DataFrame(dataAll.T, columns=features_y)
cr = dataAll2.corr(method=method)

# calculating t-statistics, based on the correlations
t_stat = (cr*(dataAll2.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll2.shape[0]-2))
#rho_long_format = cr.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'Correlation'})
#pval_long_format = pv.stack().reset_index().rename(columns={'level_0':'Feature_1','level_1':'Feature_2', 0:'pval'})
#results = pd.concat([pval_long_format, rho_long_format["Correlation"]], axis=1)


In [ ]:
cr = cr.to_numpy()
#pv = pv.to_numpy()

In [ ]:
for i in iRow:
    for j in iCol:
        if i<=j:
            #print(i, j)
            #print(dataAll[i])
            X = dataAll[i]
            Y = dataAll[j]
            nas = np.logical_or(X != X, Y != Y)
            not_na = sum(~nas)
            #X = Y[~nas]
            #Y = Y[~nas]
            #new_X, new_Y = remove_pairs_with_a_missing(X, Y)
            correlation = cr[i, j]
            pval =  pv[i, j]
            tests.append([features[i], features_y[j], pval, correlation, not_na])
results = pd.DataFrame(tests, columns=['Feature_1','Feature_2','pval', 'Correlation', 'Not_NAs'])

p_adust, p_threshold = bh(results["pval"].values, fdr)

results["P_adusted"] = p_adust
results["bh_fdr_threshold"] = p_threshold
results['Type'] = Type
results = results.sort_values(['pval', 'Correlation'],
                              ascending=[True, False])

In [ ]:
# feature names for the complete dataset
featuresAll = featuresX.copy()
featuresAll.extend(featuresY)

In [ ]:
results

In [ ]:
# creating the complete dataset
dataAll_df = pd.DataFrame(dataAll.T, columns=featuresAll)

In [ ]:
%%time
# correlation
cr = dataAll_df.corr(method='pearson')

# calculaing t-statistics, based on the correlations
t_stat = (cr*(dataAll_df.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll_df.shape[0]-2))

In [ ]:
cr

In [ ]:
%%time
# correlation
cr = dataAll_df.corr(method='pearson')

# calculaing t-statistics, based on the correlations
t_stat = (cr*(dataAll_df.shape[0]-2)**.5)/(1-cr**2)**.5

# calculating p-values based on the t-statistics
pv = 2 * (1 - t.cdf(abs(t_stat), df=dataAll_df.shape[0]-2))

In [ ]:
X_Y

In [ ]:
within_X

In [ ]:
within_Y

In [ ]:
X_Y.pval

In [ ]:
outputpath = "/Users/rah/Documents/omicsEye/btest/demo/btest_output"
os.makedirs(outputpath, exist_ok=True)
X_Y.to_csv(outputpath + '/X_Y.tsv', sep="\t")
#rho_X = rho.iloc[0:500, 0:500]
#rho_X.to_csv(outputpath + '/simtable_X.tsv', sep="\t")
#rho_X_Y.to_csv(outputpath + '/simtable.tsv', sep="\t")
within_X.to_csv(outputpath + '/within_X.tsv', sep="\t")
within_Y.to_csv(outputpath + '/within_Y.tsv', sep="\t")

In [ ]:
#associations = blockplot.load_associations(path = outputpath + '/X_Y.tsv')
associations = blockplot.load_associations('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new/X_Y.tsv')

In [ ]:
outputpath

In [ ]:
associations

In [ ]:
#simtable = blockplot.load_order_table(outputpath + '/simtable.tsv', associations)
simtable = blockplot.load_order_table('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new//simtable.tsv', associations)

In [ ]:
X_Y

In [ ]:
blockplot.plot(
    simtable,
    associations,
    cmap="RdBu_r",
    mask=False,
    axlabels=["",""],
    outfile="/Users/rah/Documents/omicsEye/btest/demo/btest_output/blockplot.pdf",
    similarity="Spearman"
)